<a href="https://colab.research.google.com/github/ankitrahejagatech/crewaimultiagentdemo/blob/main/L2_research_write_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [22]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [23]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

It will research and write an article for you

In [24]:
from crewai import Agent, Task, Crew


In [25]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/utils.py')

Mounted at /content/drive


- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [41]:
import os
import utils
from utils import get_openai_api_key
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

We need a Planner, Writer, and Editor

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [42]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [43]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [44]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [45]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [46]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [47]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.
- `verbose=1` allows you to see subset of logs of the execution.



In [48]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [49]:
result = crew.kickoff(inputs={"topic": "Agents"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agents.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Ultimate Guide to Agents: Trends, Players, and News 

Outline: 
I. Introduction
- Definition of agents in various industries
- Importance of agents in today's business landscape

II. Latest Trends in Agents
- Rise of AI-powered agents in customer service
- Increased use of virtual agents in real estate
- Growing popularity of social media influencers as agents

III. Key Players in the Agent Industry
- Top real estate agents and agencies
- Leading talent agents in the entertainment industry
- Influe

- Display the results of your execution as markdown in the notebook.

In [50]:
from IPython.display import Markdown
Markdown(result)

# The Ultimate Guide to Agents: Trends, Players, and News

In today's fast-paced business landscape, agents play a crucial role in various industries by acting as intermediaries between clients and professionals. Agents are individuals or entities that represent and negotiate on behalf of their clients, providing expertise and guidance to achieve desired outcomes. From real estate agents helping buyers find their dream homes to talent agents securing roles for actors in Hollywood, the role of agents is diverse and essential.

## Latest Trends in Agents

One of the latest trends in the agent industry is the rise of AI-powered agents in customer service. Companies are increasingly turning to artificial intelligence to enhance their customer support functions and provide timely and personalized assistance to clients. Virtual agents are also revolutionizing the real estate industry, allowing buyers and sellers to interact with properties and conduct transactions online. Moreover, social media influencers are now acting as agents, leveraging their large followings to promote products and services to their audience.

## Key Players in the Agent Industry

In the real estate sector, top agents and agencies such as Keller Williams and RE/MAX dominate the market, offering unparalleled expertise and customer service to their clients. In the entertainment industry, talent agents from agencies like CAA and WME represent A-list actors and negotiate lucrative deals for their clients. Insurance agents and brokers are also influential players in the industry, providing individuals and businesses with essential coverage and financial protection.

## Noteworthy News on Agents

Recent mergers and acquisitions have been reshaping the agent industry, with larger agencies consolidating their market share and expanding their services. The COVID-19 pandemic has also impacted agents and their work, forcing professionals to adapt to remote work environments and virtual interactions with clients. Despite these challenges, there are success stories of agents making a difference in their fields, going above and beyond to deliver exceptional results for their clients.

## Target Audience Analysis

Business professionals looking to understand the role of agents can benefit from learning about the latest trends and key players in the industry. Individuals considering a career as an agent can gain valuable insights into the various opportunities available and the skills required to succeed in the field. Consumers seeking guidance from agents in real estate, entertainment, insurance, and other industries can make informed decisions by understanding the role and impact of agents in their transactions.

## Call to Action

For those interested in exploring opportunities in the agent industry, consider researching and networking with professionals in your desired field. Resources such as industry publications, online courses, and networking events can provide valuable information and connections to help you kickstart your career as an agent. Share your own agent experiences on social media to inspire others and showcase the value you bring to your clients.

In conclusion, agents play a vital role in today's business landscape, representing the interests of their clients and driving positive outcomes in various industries. By staying informed about the latest trends, key players, and noteworthy news in the agent industry, you can position yourself for success as a professional or consumer seeking guidance from agents. Explore the opportunities available in the agent industry and make the most of your interactions with agents in your personal and professional endeavors.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).